In [13]:
import sys, plotly
print("python:", sys.executable)
print("plotly:", plotly.__version__)
try:
    import nbformat
    print("nbformat:", nbformat.__version__)
except Exception as e:
    print("nbformat import error:", e)
import plotly.io as pio
print("renderer:", pio.renderers.default)

python: c:\Users\Ivan\roboAI\.venv\Scripts\python.exe
plotly: 6.3.1
nbformat: 5.10.4
renderer: browser


In [14]:
import json, os
from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "browser"

LOG_DIR = Path("../data/logs")
files = sorted(LOG_DIR.glob("run_*.json"))
assert files, f"No logs found in {LOG_DIR}"
with open(files[-1], "r", encoding="utf-8") as f:
    data = json.load(f)

events = data.get("events", [])
df = pd.json_normalize(events)
df["t0"] = df["t"] - df["t"].min() if "t" in df.columns else range(len(df))

# Path (x,y) from spa_tick
need_cols = {"t0","x","y","left_cmd","right_cmd"}
path_df = df[df["op"] == "spa_tick"].copy()
missing = need_cols - set(path_df.columns)
if missing:
    print("Heads up: missing columns in spa_tick:", missing)
if path_df.empty:
    raise RuntimeError("No 'spa_tick' rows found. Make sure you ran the SPA controller (the planner version logs spa_tick each tick).")

# Wheel command time series
fig_cmds = go.Figure()
fig_cmds.add_trace(go.Scatter(x=path_df["t0"], y=path_df.get("left_cmd"),  name="left_cmd"))
fig_cmds.add_trace(go.Scatter(x=path_df["t0"], y=path_df.get("right_cmd"), name="right_cmd"))
fig_cmds.update_layout(title="Wheel commands vs time", xaxis_title="time (s)", yaxis_title="rad/s")
fig_cmds.show()

# Path plot (x,y)
fig_path = px.line(path_df, x="x", y="y", title="Estimated path")
fig_path.update_yaxes(scaleanchor="x", scaleratio=1)
fig_path.show()

# Front obstacle level during forward ticks
if "front" in df.columns:
    front_df = df[df["op"] == "spa_forward_tick"][["t0","front"]]
    if not front_df.empty:
        fig_front = px.line(front_df, x="t0", y="front", title="Front obstacle level (normalized)")
        fig_front.show()

# Also save HTML copies
out_dir = Path("../reports"); out_dir.mkdir(parents=True, exist_ok=True)
fig_cmds.write_html(out_dir / "wheel_cmds.html")
fig_path.write_html(out_dir / "path.html")
print(f"Loaded log: {files[-1].name} — events: {len(df)}")
print(f"Saved HTML to: {out_dir/'wheel_cmds.html'} and {out_dir/'path.html'}")


Loaded log: run_20251005_161349.json — events: 118
Saved HTML to: ..\reports\wheel_cmds.html and ..\reports\path.html
